https://stackoverflow.com/questions/63528705/how-to-modify-edge-labels-in-digraph-of-networkx

In [1]:
import numpy as np
from collections import Counter

In [10]:
edges = {'in_0out_1': ['in_0', 'out_1', -3.6377842209674958],
'mid_2mid_0': ['mid_2', 'mid_0', 0.6165115214405615],
'in_1mid_1': ['in_1', 'mid_1', 3.6368075690523423],
'in_0mid_1': ['in_0', 'mid_1', -2.76795360903403],
'mid_0out_0': ['mid_0', 'out_0', -0.7665496719059972],
'mid_0out_1': ['mid_0', 'out_1', 2.0047001373416755],
'in_2mid_2': ['in_2', 'mid_2', -1.34533801312376],
'in_mid_1_0': ['in_1', 'mid_0', 2.136059819929803],
'mid_0mid_0': ['mid_0', 'mid_0', -1.8265832443155807],
'mid_out_1_0': ['mid_1', 'out_0', 2.9330077826949488],
'in_0out_0': ['in_0', 'out_0', 2.913596825881276],
'in_1out_2': ['in_1', 'out_2', 3.5933465588280176]}

edges = [tuple(edges[i]) for i in edges]

In [11]:
d = {}
for item in edges:
    if item[1] in d:
        d[item[1]].update({item[0]:item[2]})
    else:
        d[item[1]] = {item[0]:item[2]}

In [12]:
dic = {}
for i in sorted(d):
    dic[i] = d[i] 

In [13]:
for i in dic:
    print(i)
    print(dic[i])

mid_0
{'mid_2': 0.6165115214405615, 'in_1': 2.136059819929803, 'mid_0': -1.8265832443155807}
mid_1
{'in_1': 3.6368075690523423, 'in_0': -2.76795360903403}
mid_2
{'in_2': -1.34533801312376}
out_0
{'mid_0': -0.7665496719059972, 'mid_1': 2.9330077826949488, 'in_0': 2.913596825881276}
out_1
{'in_0': -3.6377842209674958, 'mid_0': 2.0047001373416755}
out_2
{'in_1': 3.5933465588280176}


calculate weight sum

In [5]:

def out_to_weight(dic):
    for key in dic:
        print(key)
        if 'mid' in key and isinstance(dic[key], dict):
            if set(i[:2] for i in dic[key]) == set(['in']):
                dic[key] = np.tanh(sum(dic[key].values()))
                out_to_weight(dic)
                
            elif key in dic[key].keys() and Counter(i[:2] for i in dic[key])['mi'] == 1:
                dic[key] = np.tanh(sum(dic[key].values()))
                out_to_weight(dic)

        elif 'out' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key:
                    dic[key][sub_key] += dic[sub_key]
            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
            
out_to_weight(dic)            

TypeError: unsupported operand type(s) for +=: 'float' and 'dict'

In [7]:
dic

{'mid_0': 0.2999608434786799,
 'mid_1': 0.7007913892899983,
 'out_0': 0.999989545440779,
 'out_1': -0.8700106012983131,
 'out_2': 0.9984879701176493}